In [ ]:
import pandas as pd
import json
import re
import spacy
import html
from nltk.corpus import stopwords
%matplotlib inline
nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))

In [ ]:
BASE = 'C:\\Users\\basharm\\PythonJupyter\\CoVID19CodeGit\\data\\initial_data\\'

In [ ]:
def lemmatise(tweet):
    doc = nlp(tweet)
    tokens = []
    for token in doc:
        tokens.append(token.lemma_)
    return ' '.join(tokens)

tweet = u"Apple is looking at buying U.K. startup for $1 billion"
lemmatise(tweet)

In [ ]:
re1 = re.compile(r'  +')
def fixup(x):
    """ Cleans up erroroneus characters"""
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ').replace('\n', ' ').replace('\t', ' ').replace('\r', ' ').replace('rt @','@')
    return re1.sub(' ', html.unescape(x))

In [ ]:
def preprocess_a_tweet(tweet):
    tweet = ' '.join([w for w in tweet.split() if not w in stop_words])
    tweet = fixup(tweet)
    tweet = re.sub(r'^RT @\w+:', 'xxrtu', tweet).strip() # retweet unmodified
    tweet = re.sub(r'^MRT @\w+:', 'xxrtm', tweet).strip() # retweet modified
    tweet = re.sub(r'@\w+', 'xxatp', tweet).strip() # @Person occurance
    tweet = re.sub(r'http\S+', 'xxurl', tweet).strip() # url occurence
    tweet = re.sub(r'(.)\1+', r'\1\1', tweet).strip() # fix repeating characters
    tweet = lemmatise(tweet)
    return tweet.lower()
    #return tweet
tweet = 'aaaaaa Canâ€™t wait to pop out of no where with this one ðŸ¤« untill then imma stay deep in this ðŸŽ’'
preprocess_a_tweet(tweet)

In [ ]:
data = []
with open(BASE+'links_only.jsonl') as FI:
    for line in FI:
        json_data = json.loads(line)
        key_val = {}
        key_val['created_at'] = json_data['created_at']
        key_val['verified'] = json_data['user']['verified']
        key_val['name'] = json_data['user']['name']
        key_val['full_text'] = preprocess_a_tweet(json_data['full_text'])
        key_val['location'] = json_data['user']['location']
        key_val['followers_count'] = json_data['user']['followers_count']
        key_val['friends_count'] = json_data['user']['friends_count']
        data.append(key_val)


In [ ]:
df_in = pd.DataFrame(data)
df_in.head(5)

In [ ]:
df_in.to_csv(BASE+'links_only_pp.csv', encoding = 'utf8', index = None)

In [ ]:
df_in = pd.read_csv(BASE+'links_only_pp.csv', encoding = 'utf8')
df_in.head(5)

In [ ]:
df_in['created_at'] =pd.to_datetime(df_in.created_at)

In [ ]:
df_in.head()

In [ ]:
df_in = df_in.sort_values(by='created_at')
df_in.head(5)

In [ ]:
df_in = df_in.reset_index()

In [ ]:
df_in

In [ ]:
## Plot

from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
wordcloud = WordCloud().generate(re.sub("([^\x00-\x7F])+","",' '.join(list(df_in['full_text'])).replace('xxatp', "").
                                 replace('xxurl', "")))


# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()